# METRIC LEARNING


1.   Configuration
2.   Important Repositories
3.   Preparing and Loading Data
4.   Training Embedder and Classifier
5.   Observing embeddings (T-SNE, UMAP)
6.   Simaese vs Triplets



# 1. Configuration

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd "/content/drive/My Drive/MCV/Exercise 1"
!ls

/content/drive/My Drive/MCV/Exercise 1
Data  losses.py		   metrics.py  networks.py	   trainer.py
dddd  MetricProject.ipynb  mmfashion   prepare_in_shop.py  utils.py


In [9]:
# -- INSTALL MMFASHION DEPENDENCIES -- #
%cd "mmfashion"
!python setup.py install
!ls

/content/drive/My Drive/MCV/Exercise 1/mmfashion
python3: can't open file 'setup.py': [Errno 2] No such file or directory
apis  core  datasets  __init__.py  models  __pycache__	utils  version.py


# 2. Important Repositories

1.   Siamese and Triplet Networks [repo](https://github.com/adambielski/siamese-triplet)
2.   MMFashion Datasets [repo](https://github.com/open-mmlab/mmfashion/)



In [8]:
# -- IMPORTS -- #
import torch
import numpy as np
import os
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from networks import EmbeddingNet, ClassificationNet
from metrics import AccumulatedAccuracyMetric
from trainer import fit

ModuleNotFoundError: ignored

# 3. Preparing and Loading Data

Here we will use the mmfashion library to load the data and be ready to load them normally or if we want woth anchor, positive and negative samples.



In [0]:
# -- CONSTANTS -- #
img_path = os.path.join(working_dir, 'Data')
annotations_path = os.path.join(img_path, 'Anno')
batch_size = 256
class_categories = {
    'Denim': 0,
    'Jackets_Vests': 1,
    'Pants': 2,
    'Shirts_Polos': 3,
    'Shorts': 4,
    'Suiting': 5,
    'Sweaters': 6,
    'Sweatshirts_Hoodies': 7,
    'Tees_Tanks': 8,
    'Blouses_Shirts': 9,
    'Cardigans': 10,
    'Dresses': 11,
    'Graphic_Tees': 12,
    'Jackets_Coats': 13,
    'Leggings': 14,
    'Rompers_Jumpsuits': 15,
    'Skirts': 16
}
num_classes = 17

In [0]:
# -- DATASETS AND DATALOADERS -- #
train_dataset = InShopDataset(img_path=img_path,
                              img_file=os.path.join(annotations_path,'train_img.txt'),
                              label_file=os.path.join(annotations_path,'train_labels.txt'),
                              id_file=os.path.join(annotations_path,'train_id.txt'),
                              bbox_file=os.path.join(annotations_path,'train_bbox.txt'),
                              landmark_file=os.path.join(annotations_path,'train_landmarks.txt'),
                              img_size=[256, 256],
                              class_mapping=class_mapping)

query_dataset = InShopDataset(img_path=img_path,
                              img_file=os.path.join(annotations_path,'query_img.txt'),
                              label_file=os.path.join(annotations_path,'query_labels.txt'),
                              id_file=os.path.join(annotations_path,'query_id.txt'),
                              bbox_file=os.path.join(annotations_path,'query_bbox.txt'),
                              landmark_file=os.path.join(annotations_path,'query_landmarks.txt'),
                              img_size=[256, 256],
                              class_mapping=class_mapping)

cuda = torch.cuda.is_available()

train_dataloader = build_dataloader(dataset=train_dataset,
                                           imgs_per_gpu=batch_size,
                                           workers_per_gpu=1,
                                           num_gpus=1,
                                           shuffle=True)

query_dataloader = build_dataloader(dataset=query_dataset,
                                           imgs_per_gpu=batch_size,
                                           workers_per_gpu=1,
                                           num_gpus=1,
                                           shuffle=False)

# 4. Training Embedder and Classifier

In this section we will train and embedder and a classifier. This way, we will obtain and embedding which will be plotted later. 

The embedder is a CNN that uses:
*   Elemento de lista
*   Elemento de lista

The classifier is just a NLP with one layer that transforms the embedder output to a dense-softmax layer to make the classification task

In [0]:
# -- CONFIGURE EMBEDDER AND CLASSIFIER HEAD -- #
embedding_net = EmbeddingNet()
model = ClassificationNet(embedding_net, n_classes=n_classes)
if cuda:
    model.cuda()
loss_fn = torch.nn.NLLLoss()
lr = 1e-2
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 50

In [0]:
# -- TRAIN EMBEDDER AND CLASSIFIER -- #
fit(train_dataloader,
    query_dataloader,
    model,
    loss_fn,
    optimizer,
    scheduler,
    n_epochs,
    cuda,
    log_interval,
    metrics=[AccumulatedAccuracyMetric()]
)